Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
#Bashayer AlAhmadi
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.
import datetime as dt
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [2]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = "AMZN"

In [3]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

end_date = dt.datetime.now()  
start_date = dt.datetime(2020,1,1)
df = pdr.get_data_yahoo(stock_symbol, start_date, end_date)
df


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-31,1842.000000,1853.260010,1832.229980,1847.839966,1847.839966,2506500
2020-01-02,1875.000000,1898.010010,1864.150024,1898.010010,1898.010010,4029000
2020-01-03,1864.500000,1886.199951,1864.500000,1874.969971,1874.969971,3764400
2020-01-06,1860.000000,1903.689941,1860.000000,1902.880005,1902.880005,4061800
2020-01-07,1904.500000,1913.890015,1892.040039,1906.859985,1906.859985,4044900
...,...,...,...,...,...,...
2021-11-08,3523.239990,3579.000000,3487.860107,3488.979980,3488.979980,3074000
2021-11-09,3515.250000,3593.770020,3501.429932,3576.229980,3576.229980,4294900
2021-11-10,3563.870117,3605.449951,3463.090088,3482.050049,3482.050049,4027400


In [ ]:
df = df[df.columns[:4]] #Retain only the first four columns of the dataframe 
#df=df.iloc[: , :4] #Second Approach to Retain only the first four columns of the dataframe
#df=df.T.head(4).T  #third Approach to Retain only the first four columns of the dataframe
df = df.dropna()
df

In [4]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".
df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean() 
df['Open_1_Change'] = df['Open'].diff() 
df = df.dropna()
X=df
X

,Open,High,Low,Close,Adj Close,Volume,Close_15_Rolling,Open_1_Change
Date,,,,,,,,
2020-01-22,1896.089966,1902.500000,1883.339966,1887.459961,1887.459961,3216300,1883.441325,31.089966
2020-01-23,1885.109985,1889.979980,1872.760010,1884.579956,1884.579956,2484600,1885.890658,-10.979980
2020-01-24,1891.369995,1894.989990,1847.439941,1861.640015,1861.640015,3766200,1883.465991,6.260010
2020-01-27,1820.000000,1841.000000,1815.339966,1828.339966,1828.339966,3528500,1880.357324,-71.369995
2020-01-28,1840.500000,1858.109985,1830.020020,1853.250000,1853.250000,2808000,1877.048657,20.500000
...,...,...,...,...,...,...,...,...
2021-11-08,3523.239990,3579.000000,3487.860107,3488.979980,3488.979980,3074000,3402.502018,46.239990
2021-11-09,3515.250000,3593.770020,3501.429932,3576.229980,3576.229980,4294900,3411.307357,-7.989990
2021-11-10,3563.870117,3605.449951,3463.090088,3482.050049,3482.050049,4027400,3415.773356,48.620117


In [5]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where(-df["Close"].diff(-1) > 0, 1, -1)


In [6]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train, X_test, y_train, y_test = X[:index], X[index:], y[:index], y[index:]



In [7]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train, y_train)

In [8]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_))) #[1/1+e^coef] <= We call all of this a probability
 

,0,1
0,Open,[1.1919890233228332e-05]
1,High,[1.2194187254329693e-05]
2,Low,[1.2354665632928381e-05]
3,Close,[1.2076095782309988e-05]
4,Adj Close,[1.2076095782309988e-05]
5,Volume,[1.2369642660858707e-09]
6,Close_15_Rolling,[1.2830321554701285e-05]
7,Open_1_Change,[-1.0139617175127892e-06]


In [9]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test) #The actual prob between 0 and 1 justlike pct.change
predictions = logistic.predict(X_test) #the resulted Y value should be 
probabilities
#predictions
#the higher the number on right the more chane go up

array([[0.43437787, 0.56562213],
       [0.43577933, 0.56422067],
       [0.43739035, 0.56260965],
       [0.43831743, 0.56168257],
       [0.43825783, 0.56174217],
       [0.43799688, 0.56200312],
       [0.4388244 , 0.5611756 ],
       [0.43974751, 0.56025249],
       [0.44010202, 0.55989798],
       [0.44055891, 0.55944109],
       [0.43999827, 0.56000173],
       [0.43911221, 0.56088779],
       [0.43946541, 0.56053459],
       [0.44018514, 0.55981486],
       [0.43980982, 0.56019018],
       [0.43978323, 0.56021677],
       [0.44012278, 0.55987722],
       [0.43959025, 0.56040975],
       [0.43960425, 0.56039575],
       [0.44001578, 0.55998422],
       [0.44025357, 0.55974643],
       [0.44026628, 0.55973372],
       [0.44040521, 0.55959479],
       [0.44075107, 0.55924893],
       [0.44059169, 0.55940831],
       [0.44081375, 0.55918625],
       [0.43963691, 0.56036309],
       [0.4394622 , 0.5605378 ],
       [0.43848957, 0.56151043],
       [0.43829643, 0.56170357],
       [0.

In [10]:
predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [11]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

metrics.confusion_matrix(y_test, predictions) #compare between y test and the new generated y value
#[true negative and false positive]

array([[ 0, 69],
       [ 0, 69]], dtype=int64)

In [12]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

logistic.score(X_test, y_test)

0.5

In [13]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(logistic ,X_test, y_test)
cross_val.mean()

0.5296296296296296